In [39]:
import torchvision.transforms as transforms
import onnxruntime as ort
import torch
from PIL import Image
import numpy as np
import pandas as pd
import glob
import os
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [40]:
# utils

import json
from pathlib import Path
from typing import Any


def log_to_file(path: str, data: Any) -> None:
    """
    Automatically write or append Python data to a JSON file.

    - If the file doesn't exist → create and write.
    - If the file exists:
        - If it contains a JSON list → append to the list.
        - If it contains a single object → convert to a list and append.
    """
    file_path = Path(path)
    file_path.parent.mkdir(parents=True, exist_ok=True)

    try:
        if file_path.exists():
            with open(file_path, "r", encoding="utf-8") as f:
                try:
                    existing = json.load(f)
                except json.JSONDecodeError:
                    existing = []

            # Ensure existing data is a list
            if not isinstance(existing, list):
                existing = [existing]

            existing.append(data)

            with open(file_path, "w", encoding="utf-8") as f:
                json.dump(existing, f, indent=4, ensure_ascii=False)

            print(f"✅ Data appended to {file_path}")
        else:
            # File doesn't exist → create and write
            with open(file_path, "w", encoding="utf-8") as f:
                json.dump(data, f, indent=4, ensure_ascii=False)

            print(f"✅ Data written to new file {file_path}")

    except TypeError as e:
        print(f"❌ Data is not JSON serializable: {e}")
    except IOError as e:
        print(f"❌ File error: {e}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")

In [41]:
CLASS_NAMES = [
    "AKIEC",
    "BCC",
    "BEN_OTH",
    "BKL",
    "DF",
    "INF",
    "MAL_OTH",
    "MEL",
    "NV",
    "SCCKA",
    "VASC",
]

base_img_dir = "../../dataset"



In [42]:
class ONNXInference:
    def __init__(self, model_path):
        """Initialize ONNX model session."""
        self.session = ort.InferenceSession(model_path)
        self.input_names = [inp.name for inp in self.session.get_inputs()]

    def preprocess_image(self, image_path):
        """Load and preprocess image to [0,1] and BCHW format."""
        img = Image.open(base_img_dir + "/" + image_path).convert("RGB")
        img = img.resize((512, 512))
        img_array = np.array(img, dtype=np.float32) / 255.0  # scale to [0,1]
        img_array = np.transpose(img_array, (2, 0, 1))  # HWC -> CHW
        img_array = np.expand_dims(img_array, axis=0)  # add batch dimension
        return img_array

    def predict(self, image_path, age, gender, location):
        """Run inference on a single image with demographic data."""
        # print(f'🖼️ image path {image_path}')
        image_tensor = self.preprocess_image(image_path)
        gender_encoded = 1.0 if gender.lower() == "m" else 0.0
        demo_tensor = np.array(
            [[float(age), gender_encoded, float(location)]], dtype=np.float32
        )
        if len(self.input_names) >= 2:
            inputs = {
                self.input_names[0]: image_tensor,
                self.input_names[1]: demo_tensor,
            }
        else:
            inputs = {self.input_names[0]: image_tensor}
        outputs = self.session.run(None, inputs)
        probs = outputs[0].flatten()
        pred = np.argmax(probs)
        return probs, pred, CLASS_NAMES[pred]

In [ ]:
# Load dataset
# df = pd.read_csv("../../dataset/data.csv")
df = pd.read_csv("../../dataset/data.csv")
le = LabelEncoder()
df["Location"] = le.fit_transform(df["Location"])
# Find all ONNX models
model_folder = "../../models/2025-11-27"
model_paths = []
# model_paths = [
#     "../../models/2025-11-27/speechmaster/18_model118.onnx",
#     "../../models/2025-11-27/speechmaster/62_model94.onnx",
#     "../../models/2025-11-27/speechmaster/122_model123.onnx",
#     "../../models/combine/2025-11-27/18vs62_down.onnx",
#     "../../models/combine/2025-11-27/18vs122_down.onnx",
# ]
# model_paths = glob.glob(os.path.join(model_folder, "**", "*.onnx"), recursive=True)

model_paths.append("../../models/2025-11-27/speechmaster/18_model118.onnx")
model_paths.append("../../models/2025-11-27/speechmaster/62_model94.onnx")

print("Found models:", model_paths)

# Store results
results = {}
prediction_result = {}


Found models: ['../../models/2025-11-27/speechmaster/18_model118.onnx', '../../models/2025-11-27/speechmaster/62_model94.onnx', '../../models/2025-11-27/yolo_skin_11.onnx']


In [ ]:

for model_path in model_paths:
    onnx_model = ONNXInference(model_path=model_path)
    model_result = {cls: 0 for cls in CLASS_NAMES}

    res_list = []

    #select specific class
    df_test = df.copy()
    # df_test = df[df['Class'] == "MEL"]
    # Inner loop wrapped with tqdm for progress
    for _, row in tqdm(
        df_test.iterrows(), total=len(df_test), desc=f"Processing {model_path.split('/')[-1]}"
    ):
        cell = {}
        probs, pred, cls = onnx_model.predict(
            row["NewFileName"], row["Age"], row["Gender"], row["Location"]
        )
        cell["probs"] = probs
        cell["pred"] = pred
        cell["cls"] = cls
        if row["Class"] in cls:
            model_result[row["Class"]] += 1
        res_list.append(cell)

    results[model_path] = model_result
    prediction_result[model_path] = res_list

Processing yolo_skin_11.onnx: 100%|██████████| 400/400 [01:02<00:00,  6.43it/s]


In [56]:
df['Class'].value_counts()

Class
BCC        2522
NV          729
BKL         544
SCCKA       473
MEL         450
AKIEC       303
DF           52
INF          50
VASC         45
BEN_OTH      43
MAL_OTH       9
Name: count, dtype: int64

In [35]:
for model_path, res in results.items():
    print(model_path, res)


In [36]:
log_to_file("../../logs/prediction_summary.log", results)
log_to_file("../../logs/prediction_list.log", prediction_result)

✅ Data appended to ../../logs/prediction_summary.log
✅ Data appended to ../../logs/prediction_list.log


In [37]:
print(results)

{}


In [38]:
classes = list(next(iter(results.values())).keys())
models = list(results.keys())

counts_per_class = df["Class"].value_counts().to_dict()

for cls in classes:
    try:
        counts = [results[model][cls] for model in models]
        percents = [results[model][cls] / counts_per_class.get(cls, 1) for model in models]

        plt.figure(figsize=(10, 5))
        bars = plt.barh(
            [model.split("/")[-1] for model in models], percents, color="skyblue"
        )

        # Add count labels on each bar
        for bar in bars:
            width = bar.get_width()
            plt.text(
                width + 0.01,
                bar.get_y() + bar.get_height() / 2,
                str(int(width * counts_per_class[cls]) ),
                va="center",
                ha="left",
            )

        plt.xlabel(f"Total Count {counts_per_class[cls]}")
        plt.ylabel("Models")
        plt.xlim(0,1)
        plt.title(f"Counts of class '{cls}' per model")
        # plt.tight_layout()
        plt.show()
    except:
        print('This class is not there')

StopIteration: 